In [67]:
# 크롤링에 필요한 모듈 import
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 사용할 웹드라이버 경로 지정
driver = webdriver.Chrome('D:/webCrawling/chromedriver.exe')

In [68]:
# 인스타 접속하는 코드
driver.get('https://www.instagram.com')
# 링크 불러오는 걸 2초간 delay 설정. 이미지 많을 경우 로딩 시간 필요하기 때문
time.sleep(2)

In [72]:
# 인스타 검색하기 위한 url 생성
word = '사당술집' # 검색할 검색어 설정
url = insta_searching(word) # insta_searching function 사용하여 url 생성

# 로그인 후 생성한 url 접속
driver.get(url)
time.sleep(3)

# select_first function 사용하여 첫번째 게시글 열기
select_first(driver)

# 비어있는 리스트 변수 선언
result = [ ]

# 몇 개 게시글 크롤링할지 설정
target = 10

# 설정한 target만큼 수집
for i in range(target):
    try:
        data = get_content(driver) # 게시글 파싱해서 필요한 정보 수집
        result.append(data) # 미리 설정한 리스트에 저장
        move_next(driver) # 게시글 정보 저장 끝나면 다음 게시글로 넘어감
    except:
        # 페이지 오류가 발생했을 경우 2초 기다리고 다음 페이지로 이동
        time.sleep(2)
        move_next(driver)
print(result[:2])

[['.지인통해 가봤지만 다수 TV출연한 통영해물보쌈나도 가봤지만 사람많은 이유를 알겠음!!고기도먹고 회도먹고블로그도 광고가아니라 실제 먹은 후기들 보니까 믿을만하네요.단체석80석 완비 단체손님도 문제 없을듯!!..통영해물보쌈02-585-4378.회식이나 모임장소는 꼭 문의하고 와야될듯해요!!..#사당맛집 #사당역맛집 #사당회식장소 #사당역회식장소 #사당모임장소 #사당역모임장소 #사당횟집', '2020-05-26', '1,571', ' ', ['#사당맛집', '#사당역맛집', '#사당회식장소', '#사당역회식장소', '#사당모임장소', '#사당역모임장소', '#사당횟집']], ['..사당 맛집 탐구생활 🔍사당~이수에서 먹어볼만한맛집들 모았다 📌@@이거 먹고싶어! 😚...#사당맛집 #사당족발 #사당역맛집 #이수맛집 #생생정보통맛집 #천하무족 #쁘띠보노 #육촌 #스시로로 #남성집 #티에리스 #로우슬로우', '2020-05-26', '1,087', ' ', ['#사당맛집', '#사당족발', '#사당역맛집', '#이수맛집', '#생생정보통맛집', '#천하무족', '#쁘띠보노', '#육촌', '#스시로로', '#남성집', '#티에리스', '#로우슬로우']]]


In [73]:
# 크롤링 하는 데 사용할 function 정의
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

# 첫번째 게시글 여는 function 정의
def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(3)

# 페이지 파싱해서 필요한 정보만 가져오는 function 정의
def get_content(driver):
    html = driver.page_source # 접속한 페이지 정보 가져오기
    soup = BeautifulSoup(html, 'lxml') # 가져온 정보 파싱
    try:
        # 특정 태그 안에 있는 text 정보 가져오기
        content = soup.select('div.C4VMK > span')[0].text
    except:
        # 예외 처리
        content = ' '
    # content 정보 중 해시태그 정보 가져오기(정규식 사용)
    tags = re.findall(r'#[^\s#,\\]+', content)
    # content 정보 중 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # content 정보 중 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except:
        like = 0
    # content 정보 중 위치정보 가져오기
    try:
        place = soup.select('div.M30c5')[0].text
    except:
        place = ' '
    
    # 뽑아낸 데이터 list에 저장
    data = [content, date, like, place, tags]
    return data

# 다음 게시글로 이동하는 function 정의
def move_next(driver):
    # 다음 게시글로 넘어가는 화살표 태그로 찾아주기
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

In [74]:
# 크롤링하여 리스트에 저장한 데이터 엑셀로 저장
import pandas as pd

# 데이터 넣을 프레임 선언
result_df = pd.DataFrame(result)
# 데이터 컬럼명 설정
result_df.columns = ['content', 'data', 'like', 'place', 'tags']
# 데이터 저장 타입과 경로 설정
result_df.to_excel('crawling_sadang.xlsx')